## Libraries

In [1]:
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    IN_COLAB = True
except:
    IN_COLAB = False

Mounted at /content/drive


In [2]:
if IN_COLAB:
    !pip install transformers
    !pip install datasets
    !pip install evaluate
    !pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 14.1 MB/s 
     |████████████████████████████████| 7.6 MB 67.3 MB/s 
     |████████████████████████████████| 182 kB 94.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 14.9 MB/s 
     |████████████████████████████████| 132 kB 105.6 MB/s 
     |████████████████████████████████| 212 kB 98.5 MB/s 
     |████████████████████████████████| 127 kB 98.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 7.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whee

In [3]:
import os
import torch

if IN_COLAB:
    root_path = '/content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA'
else:
    root_path = '/Users/kscaria/Library/CloudStorage/GoogleDrive-scariakevin1@gmail.com/My Drive/Knowledge/MSIT/Research/InstructABSA'
    
use_mps = True if torch.has_mps else False
os.chdir(root_path)

In [4]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

from InstructABSA.utils import T5ATE
from InstructABSA.data_prep import ModelReadyData

In [5]:
rest_train_file_path = './Data/semeval14/ABSA_TrainData/Restaurants_Train_v2.csv'
laptops_train_file_path = './Data/semeval14/ABSA_TrainData/Laptop_Train_v2.csv'
rest_test_file_path = './Data/semeval14/ABSA_Gold_TestData/Restaurants_Test_Gold.csv'
laptops_test_file_path = './Data/semeval14/ABSA_Gold_TestData/Laptops_Test_Gold.csv'

# Load the data
res_tr_df = pd.read_csv(rest_train_file_path)
lap_tr_df = pd.read_csv(laptops_train_file_path)
res_te_df = pd.read_csv(rest_test_file_path)
lap_te_df = pd.read_csv(laptops_test_file_path)

In [6]:
# Extract the aspect term and polarity for each review
model_data = ModelReadyData()

# Get the input text into the required format
bos_instruction = 'Review: '
eos_instruction = ' Aspect: '
res_tr_df = model_data.create_data_in_ate_format(res_tr_df, 'term', 'raw_text', 'aspectTerms', 'labels', bos_instruction, eos_instruction)
lap_tr_df = model_data.create_data_in_ate_format(lap_tr_df, 'term', 'raw_text', 'aspectTerms', 'labels', bos_instruction, eos_instruction)
res_te_df = model_data.create_data_in_ate_format(res_te_df, 'term', 'raw_text', 'aspectTerms', 'labels', bos_instruction, eos_instruction)
lap_te_df = model_data.create_data_in_ate_format(lap_te_df, 'term', 'raw_text', 'aspectTerms', 'labels', bos_instruction, eos_instruction)

## 10% Training Data

In [ ]:
# Experimentation
experiment_id = 'combined100'
experiment_name = 'combined100_vanilla'
model_checkpoint = 't5-base'

# Create T5 utils object
t5_exp = T5ATE(model_checkpoint, experiment_id, res_tr_df, res_te_df, lap_tr_df, lap_te_df, sample_size=0.1)

if IN_COLAB:
    model_out_path = os.path.join(root_path, 'T5', 'ATE')
else:
    model_out_path = os.getcwd()

model_out_path = os.path.join(model_out_path, f"{model_checkpoint}-{experiment_name}", "checkpoints")
print('Model output path: ', model_out_path)


# Tokenize Datasets
dataset, tokenized_dataset = t5_exp.set_data_for_training_semeval(experiment_id)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'evaluation_strategy':"epoch",
    'learning_rate':5e-5,
    'per_device_train_batch_size':16,
    'per_device_eval_batch_size':16,
    'num_train_epochs':4,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'predict_with_generate':True,
    'use_mps_device':use_mps
}

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Model output path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/t5-base-combined100_vanilla/checkpoints


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'text', '__index_level_0__'],
        num_rows: 609
    })
    validation: Dataset({
        features: ['labels', 'text'],
        num_rows: 1600
    })
})

In [ ]:
# Train Model
model_trainer = t5_exp.train(tokenized_dataset, **training_args)

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 609
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 156
  Number of trainable parameters = 222903552


Trainer device: cuda:0

Model training started ....


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,1.091068
2,No log,0.553557
3,No log,0.490590
4,No log,0.460414


***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/t5-base-combined100_vanilla/checkpoints
Configuration saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/t5-base-combined100_vanilla/checkpoints/config.json
Model weights saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/t5-base-combined100_vanilla/checkpoints/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/t5-base-combined100_vanilla/checkpoints/tokenizer_config.json
Special tokens file saved in /content/drive/My

In [ ]:
# Model inference
best_model = 'checkpoints'
print('Getting model from path: ', model_out_path)

# Get prediction labels - Training set
tr_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = tokenized_dataset, sample_set = 'train')

# Get prediction labels - Testing set
te_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = tokenized_dataset, sample_set = 'validation')

The following columns in the test set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 609
  Batch size = 16


Getting model from path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/t5-base-combined100_vanilla/checkpoints


***** Running Prediction *****
  Num examples = 1600
  Batch size = 16


In [ ]:
# Add new column in the respective dataframes
tr_df = pd.DataFrame(dataset['train'])
te_df = pd.DataFrame(dataset['validation'])

tr_df['pred_labels'] =  tr_pred_labels
te_df['pred_labels'] =  te_pred_labels

# Metrics
print('Train accuracy: ', tr_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(tr_df))
print('Test accuracy: ', te_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(te_df))

#Dump outputs
dump_path = '/'.join(model_out_path.split('/')[:-1])
id_tr_filename = t5_exp.get_csv_filename(experiment_id)[0]
tr_df.to_csv(os.path.join(dump_path, '10pc_' + id_tr_filename), index = False)

id_te_filename = t5_exp.get_csv_filename(experiment_id)[1]
te_df.to_csv(os.path.join(dump_path, '10pc_' + id_te_filename), index = False)

Train accuracy:  68.80131362889983
Test accuracy:  57.0625


## 20% Training Data

In [ ]:
# Experimentation
experiment_id = 'combined100'
experiment_name = 'combined100_vanilla'
model_checkpoint = 't5-base'

# Create T5 utils object
t5_exp = T5ATE(model_checkpoint, experiment_id, res_tr_df, res_te_df, lap_tr_df, lap_te_df, sample_size=0.2)

if IN_COLAB:
    model_out_path = os.path.join(root_path, 'T5', 'ATE')
else:
    model_out_path = os.getcwd()

model_out_path = os.path.join(model_out_path, f"{model_checkpoint}-{experiment_name}", "checkpoints")
print('Model output path: ', model_out_path)


# Tokenize Datasets
dataset, tokenized_dataset = t5_exp.set_data_for_training_semeval(experiment_id)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'evaluation_strategy':"epoch",
    'learning_rate':5e-5,
    'per_device_train_batch_size':16,
    'per_device_eval_batch_size':16,
    'num_train_epochs':4,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'predict_with_generate':True,
    'use_mps_device':use_mps
}

Could not locate the tokenizer configuration file, will try to use the model config instead.


Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/a4986abc64c580966eb8dd0ed61ceafaeb0d1d45/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
  

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

loading file spiece.model from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/a4986abc64c580966eb8dd0ed61ceafaeb0d1d45/spiece.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/a4986abc64c580966eb8dd0ed61ceafaeb0d1d45/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/a4986abc64c580966eb8dd0ed61ceafaeb0d1d45/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,


Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/a4986abc64c580966eb8dd0ed61ceafaeb0d1d45/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
  

Model output path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/t5-base-combined100_vanilla/checkpoints


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'text', '__index_level_0__'],
        num_rows: 1217
    })
    validation: Dataset({
        features: ['labels', 'text'],
        num_rows: 1600
    })
})

In [ ]:
# Train Model
model_trainer = t5_exp.train(tokenized_dataset, **training_args)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1217
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 308
  Number of trainable parameters = 222903552
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a 

Trainer device: cuda:0

Model training started ....


Epoch,Training Loss,Validation Loss
1,No log,0.521222
2,No log,0.375192
3,No log,0.349848
4,No log,0.343840


***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/t5-base-combined100_vanilla/checkpoints
Configuration saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/t5-base-combined100_vanilla/checkpoints/config.json
Model weights saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/t5-base-combined100_vanilla/checkpoints/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/t5-base-combined100_vanilla/checkpoints/tokenizer_config.json
Special tokens file saved in /content/drive/My

In [ ]:
# Model inference
best_model = 'checkpoints'
print('Getting model from path: ', model_out_path)

# Get prediction labels - Training set
tr_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = tokenized_dataset, sample_set = 'train')

# Get prediction labels - Testing set
te_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = tokenized_dataset, sample_set = 'validation')

The following columns in the test set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1217
  Batch size = 16


Getting model from path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/t5-base-combined100_vanilla/checkpoints


***** Running Prediction *****
  Num examples = 1600
  Batch size = 16


In [ ]:
# Add new column in the respective dataframes
tr_df = pd.DataFrame(dataset['train'])
te_df = pd.DataFrame(dataset['validation'])

tr_df['pred_labels'] =  tr_pred_labels
te_df['pred_labels'] =  te_pred_labels

# Metrics
print('Train accuracy: ', tr_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(tr_df))
print('Test accuracy: ', te_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(te_df))

#Dump outputs
dump_path = '/'.join(model_out_path.split('/')[:-1])
id_tr_filename = t5_exp.get_csv_filename(experiment_id)[0]
tr_df.to_csv(os.path.join(dump_path, '20pc_' + id_tr_filename), index = False)

id_te_filename = t5_exp.get_csv_filename(experiment_id)[1]
te_df.to_csv(os.path.join(dump_path, '20pc_' + id_te_filename), index = False)

Train accuracy:  75.02054231717338
Test accuracy:  66.125


## 50% Training Data

In [7]:
# Experimentation
experiment_id = 'combined100'
experiment_name = 'combined100_vanilla'
model_checkpoint = 't5-base'

# Create T5 utils object
t5_exp = T5ATE(model_checkpoint, experiment_id, res_tr_df, res_te_df, lap_tr_df, lap_te_df, sample_size=0.5)

if IN_COLAB:
    model_out_path = os.path.join(root_path, 'T5', 'ATE')
else:
    model_out_path = os.getcwd()

model_out_path = os.path.join(model_out_path, f"{model_checkpoint}-{experiment_name}", "checkpoints")
print('Model output path: ', model_out_path)


# Tokenize Datasets
dataset, tokenized_dataset = t5_exp.set_data_for_training_semeval(experiment_id)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'evaluation_strategy':"epoch",
    'learning_rate':5e-5,
    'per_device_train_batch_size':16,
    'per_device_eval_batch_size':16,
    'num_train_epochs':4,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'predict_with_generate':True,
    'use_mps_device':use_mps
}

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Model output path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/t5-base-combined100_vanilla/checkpoints


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [8]:
# Train Model
model_trainer = t5_exp.train(tokenized_dataset, **training_args)

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 3043
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 764
  Number of trainable parameters = 222903552


Trainer device: cuda:0

Model training started ....


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,0.335369
2,No log,0.257649
3,1.034400,0.237152
4,1.034400,0.232572


***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/t5-base-combined100_vanilla/checkpoints
Configuration saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/t5-base-combined100_vanilla/checkpoints/config.json
Model weights saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/t5-base-combined100_vanilla/checkpoints/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/t5-base-combined100_vanilla/checkpoints/tokenizer_config.json
Special tokens file saved in /content/drive/My

In [9]:
# Model inference
best_model = 'checkpoints'
print('Getting model from path: ', model_out_path)

# Get prediction labels - Training set
tr_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = tokenized_dataset, sample_set = 'train')

# Get prediction labels - Testing set
te_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = tokenized_dataset, sample_set = 'validation')

The following columns in the test set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3043
  Batch size = 16


Getting model from path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/t5-base-combined100_vanilla/checkpoints


***** Running Prediction *****
  Num examples = 1600
  Batch size = 16


In [10]:
# Add new column in the respective dataframes
tr_df = pd.DataFrame(dataset['train'])
te_df = pd.DataFrame(dataset['validation'])

tr_df['pred_labels'] =  tr_pred_labels
te_df['pred_labels'] =  te_pred_labels

# Metrics
print('Train accuracy: ', tr_df[['pred_labels', 'labels']].apply(lambda x: len([i for i in x[1].split(', ') if i in x[0].split(', ')]) == len(x[1].split(', ')), axis=1).sum()/len(tr_df))
print('Test accuracy: ', te_df[['pred_labels', 'labels']].apply(lambda x: len([i for i in x[1].split(', ') if i in x[0].split(', ')]) == len(x[1].split(', ')), axis=1).sum()/len(te_df))

#Dump outputs
dump_path = '/'.join(model_out_path.split('/')[:-1])
id_tr_filename = t5_exp.get_csv_filename(experiment_id)[0]
tr_df.to_csv(os.path.join(dump_path, '50pc_' + id_tr_filename), index = False)

id_te_filename = t5_exp.get_csv_filename(experiment_id)[1]
te_df.to_csv(os.path.join(dump_path, '50pc_' + id_te_filename), index = False)

Train accuracy:  0.8701938876109103
Test accuracy:  0.809375
